## Examples to work through
#### Some code to get you started
(though you still need to sort out chunks yourself)
For many examples, I recommend only working with a specific region, gridpoint or time slice to speed up the calculation and save yourself time

In [1]:
import numpy as np
import xarray as xr
import time

In [2]:
from dask.distributed import Client
client = Client(threads_per_worker=1,memory_limit=0)
client.amm.start()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40533,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42283,Total threads: 1
Dashboard: /proxy/36787/status,Memory: 0 B
Nanny: tcp://127.0.0.1:34115,


In [3]:
import intake
catalog = intake.cat.access_nri

### 1. The windiest month ever in your birth town 
(using ERA5 global monthly zonal surface wind and meridional surface wind)

```python
xr.open_mfdataset('/g/data/rt52/era5/single-levels/monthly-averaged/10u/*/*.nc', # zonal wind
                  parallel=True, use_cftime=True, chunks = {}).u10
xr.open_mfdataset('/g/data/rt52/era5/single-levels/monthly-averaged/10v/*/*.nc', # meridional wind
                  parallel=True, use_cftime=True, chunks = {}).v10

# OR
catalog['era5_rt52'].search(variable='10u', # zonal wind
                            product='era5-monthly-averaged'
                           ).to_dask(xarray_open_kwargs={'chunks':{},'use_cftime':True}).u10

catalog['era5_rt52'].search(variable='10v', # meridional wind
                            product='era5-monthly-averaged'
                           ).to_dask(xarray_open_kwargs={'chunks':{},'use_cftime':True}).v10
```

### 2. The odds of it raining tomorrow given it is raining today 
(using ERA5 global hourly precipitation rate)


```python
xr.open_mfdataset('/g/data/rt52/era5/single-levels/reanalysis/tp/*/*.nc', # total precipitation in each hour
                  parallel=True, use_cftime=True, chunks = {}).tp

# OR
catalog['era5_rt52'].search(variable='tp', # total precipitation in each hour
                            product='era5-reanalysis'
                           ).to_dask(xarray_open_kwargs={'chunks':{},'use_cftime':True}).tp
```

### 3. The average temperature difference between days when it rains and days when it doesn’t 
(using ACCESS-ESM1-5 daily temperature and precipitation)

```python
xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/day/tas/gn/latest/*.nc', # daily surface air temperature
                  parallel=True, use_cftime=True, chunks={}).tas

xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/piControl/r1i1p1f1/day/pr/gn/latest/*.nc', # daily mean precipitation flux (kg/m^2/s)
                  parallel=True, use_cftime=True, chunks={}).pr                  

# OR
catalog['cmip6_fs38'].search(variable_id='tas', # daily surface air temperature
                            source_id = 'ACCESS-ESM1-5',
                             experiment_id='piControl',
                             version='latest',
                             frequency='day',
                           ).to_dask(xarray_open_kwargs={'chunks':{}, 'use_cftime':True}).tas
                           
catalog['cmip6_fs38'].search(variable_id='pr', # daily mean precipitation flux (kg/m^2/s)
                            source_id = 'ACCESS-ESM1-5',
                             experiment_id='piControl',
                             version='latest',
                             frequency='day',
                           ).to_dask(xarray_open_kwargs={'chunks':{}, 'use_cftime':True}).pr
```

### 4. EXTENSION: The rate of global warming for coastal oceans 
(i.e.  gridcells adjacent to the coast)
Leaving this a little bit more open to interpretation and choices.

```python
catalog['cmip6_fs38'].search(variable_id='tos', 
                            source_id = # 'ACCESS-ESM1-5' or 'ACCESS-CM2'
                             experiment_id = # recommend 'ssp585'
                             version='latest',
                             member_id = 'r1i1p1f1', # unless you want to use multiple ensemble members
                             frequency= # recommend 'mon'
                           ).unique().to_dask(xarray_open_kwargs={'chunks':{}, 'use_cftime':True}).tos
```